In [2]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML
import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [3]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()

0

In [4]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [5]:
from transformers import Wav2Vec2ProcessorWithLM
processor = Wav2Vec2ProcessorWithLM.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

In [6]:
import glob 
data_dir  = "/workspace/data_processed/*"
data_dir_list = glob.glob(data_dir)
data_dir_list.remove('/workspace/data_processed/5')

In [7]:
data_dir_list

['/workspace/data_processed/26',
 '/workspace/data_processed/13',
 '/workspace/data_processed/2',
 '/workspace/data_processed/7',
 '/workspace/data_processed/14',
 '/workspace/data_processed/24',
 '/workspace/data_processed/3',
 '/workspace/data_processed/38',
 '/workspace/data_processed/28',
 '/workspace/data_processed/16',
 '/workspace/data_processed/30',
 '/workspace/data_processed/36',
 '/workspace/data_processed/4',
 '/workspace/data_processed/18',
 '/workspace/data_processed/29',
 '/workspace/data_processed/8',
 '/workspace/data_processed/31',
 '/workspace/data_processed/27',
 '/workspace/data_processed/6',
 '/workspace/data_processed/10',
 '/workspace/data_processed/21',
 '/workspace/data_processed/15',
 '/workspace/data_processed/35',
 '/workspace/data_processed/9',
 '/workspace/data_processed/25',
 '/workspace/data_processed/22',
 '/workspace/data_processed/33',
 '/workspace/data_processed/20',
 '/workspace/data_processed/12',
 '/workspace/data_processed/39',
 '/workspace/data

In [8]:
data_set_list_total = []
for i in range(len(data_dir_list)):
    common_voice_val = Dataset.load_from_disk(data_dir_list[i])
    data_set_list_total.append(common_voice_val)

In [9]:
common_voice_test = concatenate_datasets(data_set_list_total[-1:])

In [10]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10000
})

In [11]:
# from transformers import AutoModelForCTC, Wav2Vec2Processor

In [12]:
processor

In [13]:
model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

In [14]:
model.to("cuda")

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), 

In [15]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10000
})

In [16]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

In [ ]:
results = common_voice_test.map(map_to_result, remove_columns=common_voice_test.column_names)

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/conda/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 110, in rebuild_cuda_tensor
    torch.cuda._lazy_init()
  File "/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py", line 204, in _lazy_init
    raise RuntimeError(
RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Process ForkPoolWorker-58:
Process ForkPoolWorker-16:
Process ForkPoolWorker-57:
Process ForkPoolWork